$H_{r}$

In [31]:
import numpy 
import random
import sys


def decToBin(n,param):
    """Convertit un nombre en binaire"""
    res = [0 for i in range(param - len(bin(n)[2:]))] + list(bin(n)[2:])
    for i in range(len(res)):
        res[i] = int(res[i])    
    return res


def matrixControl(param):
    L = [2**i for i in range(param)]
    L.reverse()
    liste = []
    for i in range(1,2**param):
        if i not in L:
            liste.append(i)
    liste = liste + L
    for i in range(len(liste)):
        liste[i] = decToBin(liste[i],param)
    H = numpy.asarray(liste)
    H = H.transpose() 
    
    return H

def matrixGen(param):
    H = matrixControl(param)
    P = H[0:param,0:2**param - param-1]
    I = numpy.eye(2**param-1- param,dtype=int)
    P = numpy.concatenate (( I, P.transpose()),axis = 1)
    
    return P
    
    


def createMessage(length):
    msg = []
    for i in range(length):
        letter = random.choice([0,1])
        msg.append(letter)
    return numpy.array(msg, dtype = int)


def encode(m, g):
    enc = numpy.dot(m, g)%2
    return enc

def encode1(m, g,param):
    res = numpy.zeros(2**param -1, dtype=int)
    for i in range(len(m)):
        if m[i] != 0:
            res = res^g[i]
            
      
    return res

def flip(m,i):
    m=list(m)
    if m[i]==0:
        m[i] = 1
    else:
        m[i] = 0
        
    return m

def decoder(H,y,param):
    
    synd = numpy.dot(y, H.transpose())%2
    if (synd == numpy.zeros(param,dtype= int)).all():
        return y
    else:
        for i in range(2**param-1):
            if (synd == H[:,i]).all():
                c = flip(y,i)
        return c
                
    
    
    
    return synd

def divise(chaine,n):
    if len(chaine)%n!=0:
        chaine=(n-len(chaine)%n)*"0"+chaine
    return [chaine[i:i+n] for i in range(0,len(chaine),n)]
    


def encodage_file(file,param):
    G = matrixGen(param)
    file_in=open(file,"r")
    chaine=file_in.read().replace('\n','')
    list_block = divise(chaine,2**param-1-param)
    file_in.close
    message_recu=""
    for block in list_block:
        block=numpy.array([int(i) for i in block], dtype=int)
        encodage = encode(block,G)
        list_block_en_chaine ="".join([str(i) for i in encodage])
        message_recu+=list_block_en_chaine
    with open("emis.txt","w",encoding="utf-8") as file_out:
        file_out.write(message_recu)
    return file_out

def StringToBin(file):
    filin = open(file,"r",encoding="utf-8")
    lines = filin.read().replace("\n","")
    lines = [bin(ord(i))[2:].zfill(8) for i in lines]
    filin.close
    with open("chaine_to_binaire.txt","w",encoding="utf-8") as filout:
        filout.write(''.join(lines))
    
    return lines



def BinToString(file):
    filin = open(file,"r",encoding="utf-8")
    lines = filin.read().replace("\n","")
    x = divise(lines,8)
    x = [chr(int(i,2)) for i in x]
    x = "".join([str(i) for i in x])
    with open("file_converted.txt","w",encoding="utf-8") as filout:
        filout.write(x)
      
    return x


def error_random(block):
    pos_random=random.randint(0,len(block)-1)
    return flip(block,pos_random)


def file_error(file,param):
    list_block=[]
    with open(file,"r",encoding="utf-8") as filin:
        lines = filin.read().replace("\n","")
    list_block = divise(lines,2**param-1)
    chaine_block_error = ''
    for block in list_block:
        block = error_random(block)
        chaine_block_error +="".join([str(i) for i in block])
    with open("file_error.txt","w",encoding="utf-8") as filout:
        filout.write(chaine_block_error)

    return chaine_block_error




def correct_file(file,param):
    H = matrixControl(param)
    file_in=open(file,"r",encoding="utf-8")
    chaine=file_in.read().replace('\n','')
    list_block = divise(chaine,2**param-1)
    file_in.close
    message_recu=""
    for block in list_block:
        block=numpy.array([int(i) for i in block], dtype=int)
        decodage = decoder(H,block,param)
        list_block_en_chaine ="".join([str(i) for i in decodage])
        message_recu+=list_block_en_chaine[0:2**param-1-param]
    with open("corrected_file.txt","w",encoding="utf-8") as file_out:
        file_out.write(message_recu)
    return file_out

In [23]:
matrixControl(2) #H

array([[1, 1, 0],
       [1, 0, 1]])

In [19]:
matrixControl(3) 

array([[0, 1, 1, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0],
       [1, 1, 0, 1, 0, 0, 1]])

In [20]:
matrixGen(2)

array([[1, 1, 1]])

In [5]:
matrixGen(3) #G

array([[1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1]])

H n'a pas de colonne nulle ni de colonne identique donc d >= 3

In [30]:
a=bin(0x26)[2:].zfill(8)+bin(0xE7)[2:].zfill(8)+bin(0x56)[2:].zfill(8)+bin(0x7F)[2:].zfill(8)
b=bin(0x79)[2:].zfill(8)+bin(0xAE)[2:].zfill(8)+bin(0xF5)[2:].zfill(8)+bin(0x18)[2:].zfill(8)

In [31]:
print(a)
print(b)

00100110111001110101011001111111
01111001101011101111010100011000


In [13]:
StringToBin('test.txt')

['01000001',
 '01000010',
 '00110001',
 '01000011',
 '00110000',
 '00110001',
 '00110000']

In [33]:
encodage_file('chaine_to_binaire.txt',3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='cp1252'>

In [34]:
BinToString('emis.txt')

'\x12ÕJ¨°\x05FU*K1Q\x92À\x12\x96J=.\x94¥\x92\x8f'

In [35]:
file_error('emis.txt',3)

'01001011000101010010101010100010100000000000010100011000010100001010100100101100110001000100011001001011000000000100001000011001001010001101000010110010010100101000011001001010001111'

In [36]:
correct_file('file_error.txt',3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='cp1252'>

In [37]:
!echo 

Commande ECHO activ‚e.


In [38]:
BinToString('corrected_file.txt')

'JE SUIS BAMBA'

In [15]:
chr(0x02)

'\x02'

In [6]:
encodage_file("source.txt",3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='cp1252'>

In [14]:
correct_file('chaine_to_binaire.txt',3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='cp1252'>

In [18]:
decoder(matrixControl(3),["1010110"],3)

ValueError: shapes (1,) and (7,3) not aligned: 1 (dim 0) != 7 (dim 0)

['1', '0', '1', '0', '1', '1', '0']

In [13]:
StringToBin('test.txt')

['01000010',
 '01000001',
 '01001101',
 '01000010',
 '01000001',
 '00100000',
 '01000100',
 '01001001',
 '01000001',
 '01000111',
 '01001110',
 '01000101']

In [14]:
encodage_file('chaine_to_binaire.txt',3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='cp1252'>

In [24]:
correct_file('test.txt',3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='utf-8'>

In [22]:
BinToString('corrected_file.txt')

'\x00XÖàÖÖxº'

# 1) Codons le message 010110010111 

In [3]:
encodage_file('source.txt',3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='utf-8'>

# Le message codé est 010101010011000111100

# 2) Decodons le message chiffré 010001110010101101001

In [4]:
correct_file('source.txt',3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='utf-8'>

# Le message clair est 011010111101

# Y'a t'ils des erreurs dans 1010110,0110011

## Pour 1010110

In [6]:
correct_file("source.txt",3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='utf-8'>

In [21]:
decoder(matrixControl(3),numpy.array([0,1,1,0,0,1,1]),3)%2

array([0, 1, 1, 0, 0, 1, 1], dtype=int32)

In [24]:
numpy.dot(numpy.array([0,1,1,0,0,1,1]),matrixControl(3).transpose())%2

array([0, 0, 0], dtype=int32)

# Il y a des erreurs dans le code 1010110 car 0010 n'est pas un prefixe de 1010110

## Pour 0110011

In [9]:
correct_file("source.txt",3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='utf-8'>

In [10]:
decoder(matrixControl(3),numpy.array([0,1,1,0,0,1,1]),3)

array([0, 1, 1, 0, 0, 1, 1])

## Il y a pas d'erreurs dans le code 0110011 car 0110 n'est pas un prefixe de 0110011

# Codons le message KHADIM NIASSE(avec espace)

In [26]:
StringToBin('source.txt')

['01001011',
 '01001000',
 '01000001',
 '01000100',
 '01001001',
 '01001101',
 '00100000',
 '01001110',
 '01001001',
 '01000001',
 '01010011',
 '01010011',
 '01000101']

In [27]:
encodage_file('chaine_to_binaire.txt',3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='utf-8'>

# Le message codé est 01001011011010010010110000110100101000111101001010100101010010110011000100101110100100101100000000010010111100000100101100110001001010001111010101000110010101010001100101001010101010

# Retrouvons le message M en enlevant les 3 derniers bits de M

In [45]:
correct_file("source.txt",3)

<_io.TextIOWrapper name='corrected_file.txt' mode='w' encoding='utf-8'>

In [46]:
BinToString('corrected_file.txt')

'Mr Brice'

['01001101',
 '01010010',
 '00100000',
 '01000010',
 '01010010',
 '01001001',
 '01000011',
 '01000101']

In [42]:
encodage_file('chaine_to_binaire.txt',3)

<_io.TextIOWrapper name='emis.txt' mode='w' encoding='utf-8'>

False

In [44]:
mes1="0100101110100101010100010110001011000000000100101001011001010100010110010010110011000100101001100101001010101010"
mes2="0100101110100101111000010110001011000000000100101001011001111000010110011001110011000110011001100101100110101010"
for i in range(len(mes1)):
    if(mes1[i]==mes2[i]):
        print(True)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
